# Análisis de series temporales
Les dejo algunos ejemplos de como podrían utilizar los pipelines para el cuarto práctico.
Es muy importante la **claridad** del código, traten de evitar hacer funciones confusas, dejando cada paso registrado y explicado.
## Definiciones

In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler

plt.rcParams['figure.figsize'] = (12,8)

holidays = './holidays.csv'
cols = ['service',
        'sender_zipcode',
        'receiver_zipcode',
        'sender_state',
        'receiver_state',
        'shipment_type',
        'quantity',
        'status',
        'date_created',
        'date_sent',
        'date_visit',
        'target']
data_path = '../datos/shipments_BR_201903.csv'

## Datos
Vamos a cargar datos de envíos que llegaron en el mes de Marzo de 2019.

In [5]:
df = pd.read_csv(data_path, usecols=cols)
df.shape

(1000000, 12)

In [6]:
df.sample(5)

,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,target
368054,SP,18078,PR,81720,express,1,0,done,2019-03-11 00:00:00,2019-03-12 12:14:00,2019-03-18 09:38:00,4
122666,MG,35519,SP,5303,standard,1,1,done,2019-03-10 00:00:00,2019-03-12 07:49:00,2019-03-16 07:52:00,4
913554,SP,9220,SP,9571,express,2,0,done,2019-03-04 00:00:00,2019-03-04 15:47:00,2019-03-05 13:11:00,1
348145,SP,9080,DF,71065,express,1,9,done,2019-03-05 00:00:00,2019-03-07 05:01:49,2019-03-09 17:50:00,1
692272,PR,87360,PE,55195,standard,1,1,done,2019-02-22 00:00:00,2019-02-25 14:19:00,2019-03-26 13:25:00,20


Vamos a tratar de clasificar los envíos como rápidos y lentos para simplificar la problemática, por lo tanto necesitamos definir un nuevo **target binario**.

Vamos a definir el nuevo target como 0 en los casos donde sea menor o igual que 3 y como 1 en los casos donde sea mayor que 3.

In [7]:
df['fast'] = (df.target <= 3).astype(np.int)
df.sample(5)

,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,target,fast
304772,SP,14140,SP,5445,express,3,0,done,2019-03-10 00:00:00,2019-03-11 15:49:00,2019-03-14 11:55:00,3,1
75038,SP,12230,PA,68507,standard,1,1,done,2019-02-19 00:00:00,2019-02-25 18:09:00,2019-03-20 13:46:00,16,0
851522,RS,90230,MG,30662,standard,2,1,done,2019-03-13 00:00:00,2019-03-14 14:16:00,2019-03-22 13:33:00,6,0
88620,SP,12341,SP,9820,express,1,0,done,2019-03-03 00:00:00,2019-03-04 17:47:00,2019-03-06 13:59:00,2,1
406204,SP,14070,MG,36320,standard,1,1,done,2019-03-06 00:00:00,2019-03-06 19:16:00,2019-03-28 09:24:00,15,0


Vamos a definir los features y el target que vamos a utilizar para entrenar y testear

In [8]:
features = ['sender_zipcode', 'receiver_zipcode', 'service']
target = 'fast'

Ahora vamos a crear las particiones de datos, quedandonos con los envíos que llegaron hasta la fecha de `cut_off` para el conjunto de entrenamiento, y con los envíos que se crearon después de la fecha de `cut_off` para el conjunto de test.

La fecha de `cut_off` la vamos a elegir en algún punto donde tengamos más del 50% de los envíos para el training, pero sin que queden demasiado pocos ejemplos para testear.

In [10]:
cut_off = '2019-03-20'
df_train = df.query(f'date_visit <= "{cut_off}"')
df_test = df.query(f'date_created > "{cut_off}"')

X_train = df_train[features].values.astype(np.float)
y_train = df_train[target].values

X_test = df_test[features].values.astype(np.float)
y_test = df_test[target].values

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((673645, 3), (673645,), (76378, 3), (76378,))

Con esta partición perdemos parte de la información, pero aún así nos queda buena cantidad de datos para trabajar en nuestros modelos.

## Logistic Regression con target binario
Vamos a pasar al diseño de nuestro modelo, lo vamos a definir como un Pipeline que en el primer paso normaliza los features utilizando `MinMaxScaler`, y en el segundo paso aplica una `LogisticRegression`.

In [12]:
model = Pipeline([
    ('normalizer', MinMaxScaler()),
    ('classifier', LogisticRegression(solver='lbfgs', 
                                      multi_class='auto')),
])

Ahora vamos a entrenar nuestro Pipeline con los datos del conjunto de **train**

In [13]:
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('normalizer', MinMaxScaler(copy=True, feature_range=(0, 1))), ('classifier', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

Y ahora vamos a generar las predicciones utilizando el conjunto de **test**

In [14]:
y_pred = model.predict(X_test)

Ahora que tenemos las predicciones de nuestro modelos, estamos en condiciones de calcular las métricas de interés

In [15]:
metrics = {
    'accuracy': accuracy_score(y_test, y_pred),
    'precision': precision_score(y_test, y_pred),
    'recall': recall_score(y_test, y_pred),
    'f1_score': f1_score(y_test, y_pred),
}

metrics

{'accuracy': 0.7165021341223913,
 'precision': 0.9277931161420946,
 'recall': 0.7397889546061611,
 'f1_score': 0.8231931867360188}

## Logistic Regression con target multiclase
Vamos a utilizar el mismo modelo que definimos anteriormente para el target binario, pero vamos a ajustar detalles para poder predecir multiclase. Esto podemos hacerlo gracias a que definimos nuestra regresión lineal con el parámetro `multiclass` en automático.

Vamos a utilizar los mismos features, cambiando únicamente el target con el que entrenamos y medimos el modelo.

In [16]:
target = 'target'
y_train = df_train[target].values
y_test = df_test[target].values
y_train.shape, y_test.shape

((673645,), (76378,))

Entrenamos otra instancia de nuestro modelo, con el target multiclase

In [ ]:
model.fit(X_train, y_train)

En esta etapa podemos ver un warning que indica que el solver que elegimos, no logró converger por lo tanto nos recomienda incrementar el número de iteraciones, es claro que las métricas obtenidas van a ser bajas, por lo tanto necesitamos modificar el pipeline. Esto no lo vamos a hacer en este notebook, sino que vamos a generar las predicciones y ver que resultados obtuvimos.

In [ ]:
y_pred = model.predict(X_test)

Por último calculamos las métricas, teniendo en cuenta que algunas métricas que utilizamos están pensadas para targets binarios. Por lo tanto necesitamos promediar los resultados por cada clase, esto ya está implementado en `scikit-learn`, y se puede hacer agregando el parámetro `average` a estas funciones.

In [45]:
metrics = {
    'accuracy': accuracy_score(y_test, y_pred),
    'precision': precision_score(y_test, y_pred, average='macro'),
    'recall': recall_score(y_test, y_pred, average='macro'),
    'f1_score': f1_score(y_test, y_pred, average='macro'),
}

metrics

/Users/rmedina/.pyenv/versions/3.7.0/envs/shipping37/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/rmedina/.pyenv/versions/3.7.0/envs/shipping37/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/rmedina/.pyenv/versions/3.7.0/envs/shipping37/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/rmedina/.pyenv/versions/3.7.0/envs/shipping37/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in l

{'accuracy': 0.432153237843358,
 'precision': 0.13690284921352877,
 'recall': 0.10089862361804243,
 'f1_score': 0.10225490576898569}

Como pensabamos, las métricas obtenidas son muy bajas. Además tenemos algunos warnings que nos avisa que algunas de las clases no tienen ejemplos, por lo tanto la métrica se define en 0 para esa clase, afectando aún más a la métrica promedio (que es la que finalmente estamos imprimiendo).